In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import sys
import time
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import pickle as pkl
import networkx as nx
import matplotlib.pyplot as plt

import src
from src.reload import deep_reload

Please run make_data_structure.sh prior to this

In [2]:
'''
Making the destinations graph
'''

gdf_place = gpd.read_file('Data/Places/tl_2023_06_place.shp')

df_place = pd.read_excel(
    'Data/Places/SUB-IP-EST2022-POP-06.xlsx',
    skiprows = 3, skipfooter = 5,
)

df_place = df_place.rename(columns = {'Unnamed: 0': 'Name', 'Unnamed: 1': 'Base'})

df_place_names = df_place['Name'].to_numpy()
df_place_population = df_place[2022].to_numpy()
gdf_place_names = gdf_place['NAME'].to_numpy()

In [3]:
pop = {}

for name in gdf_place_names:

    for idx, check_name in enumerate(df_place_names):

        if name in check_name:

            pop[name] = df_place_population[idx]
            break

populations = []

for name in gdf_place_names:

    populations.append(pop.get(name, 0))

gdf_place['population'] = populations
gdf_place['log_population'] = np.log(gdf_place['population'])

In [5]:
gdf_place_sel = gdf_place[gdf_place['population'] > 1e3]
lon, lat = np.array([x.coords.xy for x in gdf_place_sel.geometry.centroid]).T[0]
names = gdf_place_sel['NAME'].to_numpy()
pop = gdf_place_sel['population'].to_numpy()

nodes = []

for idx in range(len(names)):

    node = {
        'id': names[idx],
        'x': lon[idx],
        'y': lat[idx],
        'population': pop[idx],
        'type': 'place',
    }

    nodes.append(node)

links = []

for idx_s in range(len(names)):
    for idx_t in range(len(names)):

        link = {
            'source': names[idx_s],
            'target': names[idx_t],
            'distance': src.utilities.haversine(
                lon[idx_s], lat[idx_s], lon[idx_t], lat[idx_t],
            )
        }

        links.append(link)
    
places = src.graph.graph_from_nlg({'nodes': nodes, 'links': links})

In [6]:
places.number_of_nodes(), places.number_of_edges()

(479, 114960)

In [7]:
for source, adj in places._adj.items():
    for target, edge in adj.items():

        edge['inverse_distance'] = np.exp(-edge['distance'] / 10e3)

c = nx.community.greedy_modularity_communities(
    places, weight = 'inverse_distance', resolution = 1,
)

In [8]:
def graph_from_communities(graph, communities):

    _node = graph._node

    nodes = []

    for idx, community in enumerate(communities):

        x_coordinates = []
        y_coordinates = []
        populations = []

        for source in community:

            x_coordinates.append(_node[source]['x'])
            y_coordinates.append(_node[source]['y'])
            populations.append(_node[source]['population'])

        node = {
            'id': f'community_{idx}',
            'x': np.mean(x_coordinates),
            'y': np.mean(y_coordinates),
            'population': sum(populations),
            'places': [k for k in community],
        }

        nodes.append(node)

    links = []

    return src.graph.graph_from_nlg({'nodes': nodes, 'links': links})

communities = graph_from_communities(places, c)

In [9]:
communities.number_of_nodes(), communities.number_of_edges()

(34, 0)

In [10]:
filename = 'additional_places.json'

additional_places = src.graph.graph_from_json(filename)

In [11]:
graph_places = nx.union(communities, additional_places)

In [24]:
src.graph.graph_to_json(graph_places, 'Outputs/places.json')

In [12]:
places.number_of_nodes(), graph_places.number_of_nodes()

(479, 38)

In [13]:
with open('Data/AFDC/evse_stations.json', 'r') as file:
    evse = json.load(file)

In [14]:
stations_raw = []

for station in evse['fuel_stations']:

    checks = (
        station['state'] == 'CA',
        station['ev_dc_fast_num'] is not None,
        station['access_code'] == 'public',
        station['status_code'] == 'E',
    )

    if all(checks):

        stations_raw.append(station)

In [15]:
'''
Merging equipment labeled as individual stations but actually forming a single station
'''

longitude = np.array([station['longitude'] for station in stations_raw])
latitude = np.array([station['longitude'] for station in stations_raw])
ids = np.array([station['id'] for station in stations_raw])
networks = np.array([station['ev_network'] for station in stations_raw])

x_s, x_f = np.meshgrid(longitude, longitude, indexing  = 'ij')
y_s, y_f = np.meshgrid(latitude, latitude, indexing  = 'ij')

distance = src.utilities.haversine(x_s, y_s, x_f, y_f)

In [16]:
def none_then_zero(x):

    if x is None:

        return 0

    else:

        return x

stations_merged = [station.copy() for station in stations_raw]

drop = []

threshold = 100

for idx, row in enumerate(distance):

    if ids[idx] in drop:

        continue

    for col, dist in enumerate(row):

        if (dist <= threshold) & (idx != col) & (networks[idx] == networks[col]):

            drop.append(ids[col])

            stations_merged[idx]['ev_dc_fast_num'] = (
                none_then_zero(stations_raw[idx]['ev_dc_fast_num']) +
                none_then_zero(stations_raw[col]['ev_dc_fast_num'])
            )

            stations_merged[idx]['ev_level2_evse_num'] = (
                none_then_zero(stations_raw[idx]['ev_level2_evse_num']) +
                none_then_zero(stations_raw[col]['ev_level2_evse_num'])
            )

In [17]:
stations_merged = [station for station in stations_merged if station['id'] not in drop]

In [18]:
len(stations_merged), len(stations_raw)

(1689, 2149)

In [19]:
df_stations = pd.DataFrame.from_dict(stations_merged)

In [20]:
node_attributes = {
    'x': 'lambda n: n["longitude"]',
    'y': 'lambda n: n["latitude"]',
    'n_dcfc': 'lambda n: n["ev_dc_fast_num"]',
    'n_acl2': 'lambda n: n["ev_level2_evse_num"]',
    'network': 'lambda n: n["ev_network"]',
    'name': 'lambda n: n["station_name"]',
    'address': 'lambda n: n["street_address"]',
    'city': 'lambda n: n["city"]',
    'state': 'lambda n: n["state"]',
    'zip': 'lambda n: n["zip"]',
    'access_code': 'lambda n: n["access_code"]',
    'status_code': 'lambda n: n["status_code"]',
    'type': 'lambda n: "station"',
}

nlg = src.graph.nlg_from_dataframe(df_stations, node_attributes)

graph_station = src.graph.graph_from_nlg(nlg)

mapping = {n: f'station_{n}' for n in graph_station.nodes}

graph_station = nx.relabel_nodes(graph_station, mapping)

In [21]:
graph_empty = nx.union(graph_places, graph_station)

In [22]:
graph_empty.number_of_nodes(), graph_empty.number_of_edges()

(1727, 0)

In [23]:
src.graph.graph_to_json(graph_empty, 'Outputs/graph_empty.json')